In [9]:
import pandas as pd # conda install pandas
import altair as alt # conda install -c conda-forge altair vega_datasets

COMPILERS = ["clang", "popcorn", "unasl"]
BENCHMARKS = ["is", "bt", "ft", "cg", "ep"]
SECTIONS = ['.text', '.data', '.rodata', '.stack_transform', '.stackmaps', '.debug']


def prep_df(df, name):
    df = df.stack().reset_index()
    df.columns = ['c1', 'c2', 'values']
    df['Section'] = name
    return df


def parse_sizes():
    sections = {}
    for compiler in COMPILERS:
        sections[compiler] = {}
        for benchmark in BENCHMARKS:
            sections[compiler][benchmark] = {}
            with open('./sizes/' + compiler + '/' + benchmark + '.txt', 'r') as reader:
                lines = reader.readlines()
                lines = [line.split() for line in lines]
                lines = [line for line in lines if (len(line) > 0 and line[0].startswith('.'))]

            sections[compiler][benchmark] = {line[0]: int(line[1]) for line in lines}

            data = [v for k, v in sections[compiler][benchmark].items() if k.startswith('.data')]
            sections[compiler][benchmark]['.data'] = sum(data)

            stack_transform = [v for k, v in sections[compiler][benchmark].items() if k.startswith('.stack_transform')]
            sections[compiler][benchmark]['.stack_transform'] = sum(stack_transform)

            debug = [v for k, v in sections[compiler][benchmark].items() if k.startswith('.debug')]
            sections[compiler][benchmark]['.debug'] = sum(debug)

            rodata = [v for k, v in sections[compiler][benchmark].items() if k.startswith('.rodata')]
            sections[compiler][benchmark]['.rodata'] = sum(rodata)

            sections[compiler][benchmark]['.stackmaps'] = sections[compiler][benchmark].get('.llvm_pcn_stackmaps', 0)

            sections[compiler][benchmark] = {k: v for (k, v) in sections[compiler][benchmark].items() if k in SECTIONS}

    return sections


sizes = parse_sizes()

sections = {}
for section in SECTIONS:
    sections[section] = pd.DataFrame(
        [[sizes[compiler][benchmark][section] for compiler in COMPILERS] for benchmark in BENCHMARKS],
        index=[name.upper() for name in BENCHMARKS],
        columns=[name.capitalize() for name in COMPILERS])
    sections[section] = prep_df(sections[section], section)

df = pd.concat(sections.values())

COLORS = ['Gold', 'LightPink', 'MidnightBlue', 'MediumVioletRed', 'LightSteelBlue', 'MediumSlateBlue']

alt.Chart(df).mark_bar().encode(
    x=alt.X('c2:N', title=None),
    y=alt.Y('sum(values):Q', axis=alt.Axis(grid=False, title="Size")),
    column=alt.Column('c1:N', title="Benchmark"),
    color=alt.Color('Section:N', scale=alt.Scale(range=COLORS, ),)).configure_view(strokeOpacity=0).properties(width=80, height=600)


alt.Chart(...)

In [10]:
import json

print(json.dumps(sizes, indent=4, sort_keys=True))

{
    "clang": {
        "bt": {
            ".data": 256,
            ".debug": 278068,
            ".rodata": 7260,
            ".stack_transform": 40,
            ".stackmaps": 80,
            ".text": 126860
        },
        "cg": {
            ".data": 256,
            ".debug": 188261,
            ".rodata": 5392,
            ".stack_transform": 40,
            ".stackmaps": 80,
            ".text": 39224
        },
        "ep": {
            ".data": 256,
            ".debug": 185182,
            ".rodata": 5184,
            ".stack_transform": 40,
            ".stackmaps": 80,
            ".text": 34888
        },
        "ft": {
            ".data": 256,
            ".debug": 208715,
            ".rodata": 8868,
            ".stack_transform": 40,
            ".stackmaps": 80,
            ".text": 52456
        },
        "is": {
            ".data": 624,
            ".debug": 170244,
            ".rodata": 4736,
            ".stack_transform": 40,
            ".stackmaps":